In [61]:
import pandas as pd
excel_data = pd.read_excel('C:/Users/msi/Downloads/DATA EUROMILLION CHOIX DISCRET (1).xlsx')

In [62]:
# les varaibles exogéne utilisée
liste_columns=['JOUR tirage','E1','E2','pair','impair','pair_E1','sum_b','is_sumpaire','corr']

In [63]:
# Importation des bibliothèques nécessaires
import pandas as pd
import statsmodels.api as sm


# Fonction pour catégoriser la varaible endogéne
def categorize_and_count(row):
    count = 0
    for number in row:
        count = number + count
    if count <=50:
      return 0
    if  50 < count <= 100:
      return 1 
    if  100 < count <= 150:
          return 2 
    if  150 < count <= 200:
          return 3 
    else: 
        return 4

In [64]:
excel_data_vendredi= excel_data[excel_data["JOUR tirage"].isin(["VENDREDI", "VE"])]

In [65]:
# Application de la fonction aux colonnes B1 à B5 pour créer la variable endogène
excel_data_vendredi["Endogéne"] = excel_data_vendredi[['B1', 'B2', 'B3', 'B4', 'B5']].apply( lambda row: categorize_and_count(row), axis=1)

C:\Users\msi\AppData\Local\Temp\ipykernel_42296\1321758062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_data_vendredi["Endogéne"] = excel_data_vendredi[['B1', 'B2', 'B3', 'B4', 'B5']].apply( lambda row: categorize_and_count(row), axis=1)


In [66]:
def categorize_étoile(row):
    count = 0
    for number in row:
        count = number + count
    if count <=10:
      return 0 
    else: 
        return 2

In [67]:
excel_data_vendredi['Groupe_E1_E2']= excel_data_vendredi[['E1', 'E2']].apply( lambda row: categorize_étoile(row), axis=1)

C:\Users\msi\AppData\Local\Temp\ipykernel_42296\1307904517.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_data_vendredi['Groupe_E1_E2']= excel_data_vendredi[['E1', 'E2']].apply( lambda row: categorize_étoile(row), axis=1)


In [68]:
# Sélection de quelques variables exogènes pour le modèle
Exogéne_varaibles = excel_data_vendredi[['Groupe_E1_E2','pair','impair','pair_E1','sum_b','is_sumpaire','corr','MAX','MIN','produit']]
endogenous_variable = excel_data_vendredi["Endogéne"]

In [69]:
# Ajout d'une colonne constante aux variables exogènes pour le terme d'interception
X = sm.add_constant(Exogéne_varaibles)


In [70]:
# Création et ajustement du modèle de régression de Poisson avec statsmodels
poisson_model = sm.GLM(endogenous_variable, X, family=sm.families.Poisson())
poisson_results = poisson_model.fit()

# Affichage du résumé du modèle
poisson_results_summary = poisson_results.summary()

In [71]:
# Affichage du résumé du modèle
poisson_results_summary 

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Endogéne   No. Observations:                 1031
Model:                            GLM   Df Residuals:                     1021
Model Family:                 Poisson   Df Model:                            9
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1354.3
Date:                Sun, 07 Jan 2024   Deviance:                       51.020
Time:                        15:42:25   Pearson chi2:                     49.4
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1584
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0660      0.024     -2.788      0.005      -0.112      -0.020
Groupe_E1_E2     0.0058      0.022      0.260      0.795      -0.038       0.050
pair            -0.1633      0.061     -2.691      0.007      -0.282      -0.044
impair          -0.1665      0.059     -2.803      0.005      -0.283      -0.050
pair_E1          0.0134      0.044      0.304      0.761      -0.073       0.100
sum_b            0.0112      0.002      7.215      0.000       0.008       0.014
is_sumpaire     -0.0016      0.044     -0.037      0.970      -0.088       0.084
corr             0.1021      0.310      0.329      0.742      -0.505       0.710
MAX              0.0007      0.005      0.142      0.887      -0.009       0.011
MIN              0.0042      0.006      0.732      0.464      -0.007       0.015
produit      -4.388e-09   2.52e-09     -1.739      0.082   -9.33e-09    5.58e-10
================================================================================
"""

Le modèle a été ajusté sur 1031 observations, avec 9 variables indépendantes.  

La log-vraisemblance est de -1354.3, indiquant la qualité de l'ajustement du modèle.

Les variables significatives (au seuil de 5 %) sont 'const', 'pair', 'impair', et 'sum_b':

'const' a un coefficient de -0.0660, suggérant une influence négative sur la variable endogène.

'pair' et 'impair' ont des effets négatifs, avec des coefficients de -0.1633 et -0.1665 respectivement.

'sum_b' a un effet positif avec un coefficient de 0.0112.

Les autres variables ('Groupe_E1_E2', 'pair_E1', 'is_sumpaire', 'corr', 'MAX', 'MIN', et 'produit') ne sont pas statistiquement significatives, comme en témoignent leurs p-values élevées.

Le pseudo R-carré de 0.1584 suggère que le modèle explique une proportion modérée de la variance dans la variable endogène.

En résumé, ce modèle Poisson identifie des relations significatives entre certaines variables prédicteurs et la variable endogène, tout en indiquant que d'autres prédicteurs n'ont pas d'influence statistiquement significative.

In [72]:
# Sélection de quelques variables exogènes pour le modèle
Exogéne_varaibles = excel_data_vendredi[['pair','impair','sum_b']]

In [73]:
# Ajout d'une colonne constante aux variables exogènes pour le terme d'interception
X = sm.add_constant(Exogéne_varaibles)

In [74]:
# Création et ajustement du modèle de régression de Poisson avec statsmodels
poisson_model = sm.GLM(endogenous_variable, X, family=sm.families.Poisson())
poisson_results = poisson_model.fit()

# Affichage du résumé du modèle
poisson_results_summary = poisson_results.summary()

In [75]:
poisson_results_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Endogéne   No. Observations:                 1031
Model:                            GLM   Df Residuals:                     1028
Model Family:                 Poisson   Df Model:                            2
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1356.5
Date:                Sun, 07 Jan 2024   Deviance:                       55.559
Time:                        15:42:25   Pearson chi2:                     53.3
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1547
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0423      0.008     -5.525      0.000      -0.057      -0.027
pair          -0.1040      0.022     -4.635      0.000      -0.148      -0.060
impair        -0.1073      0.021     -5.109      0.000      -0.149      -0.066
sum_b          0.0097      0.001     13.091      0.000       0.008       0.011
==============================================================================
"""

In [76]:
# Calcul des effets marginaux
marginal_effects = poisson_results.get_margeff()
print(marginal_effects.summary())

         GLM Marginal Effects        
Dep. Variable:               Endogéne
Method:                          dydx
At:                           overall
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
pair          -0.2113      0.045     -4.697      0.000      -0.299      -0.123
impair        -0.2181      0.042     -5.189      0.000      -0.300      -0.136
sum_b          0.0198      0.002     12.586      0.000       0.017       0.023


Les résultats des effets marginaux de votre modèle GLM indiquent :


pair :
Effet marginal : -0.2113
P-value : <0.0001
Cela signifie qu'une unité d'augmentation dans 'pair' entraîne une diminution de 0.2113 unités dans la probabilité moyenne de la variable 'Endogéne'. La signification statistique (P < 0.0001) souligne l'importance de 'pair' dans le modèle.


impair :
Effet marginal : -0.2181
P-value : <0.0001
Semblable à 'pair', une unité d'augmentation dans 'impair' est associée à une diminution de 0.2181 unités dans la probabilité moyenne de 'Endogéne'. La forte signification indique également l'impact significatif de 'impair'.


sum_b :
Effet marginal : 0.0198
P-value : <0.0001
En contraste avec 'pair' et 'impair', 'sum_b' a un effet positif. Chaque unité supplémentaire augmente la probabilité moyenne de 'Endogéne' de 0.0198 unités. La signification statistique élevée montre que 'sum_b' est un prédicteur clé.



En conclusion, 'pair' et 'impair' réduisent la probabilité de la variable 'Endogéne', tandis que 'sum_b' l'augmente. Les faibles p-values indiquent que ces effets sont statistiquement significatifs. Cela suggère que ces variables jouent un rôle crucial dans la prédiction de 'Endogéne' dans votre modèle.







In [77]:
X.describe()

,const,pair,impair,sum_b
count,1031.0,1031.000000,1031.000000,1031.000000
mean,1.0,2.462658,2.537342,127.222114
std,0.0,1.061375,1.061375,29.565890
min,1.0,0.000000,0.000000,45.000000
25%,1.0,2.000000,2.000000,107.000000
50%,1.0,2.000000,3.000000,128.000000
75%,1.0,3.000000,3.000000,147.000000
max,1.0,5.000000,5.000000,217.000000


In [78]:
import pandas as pd

# Création de la nouvelle observation avec les valeurs fournies
new_observation = {
    'const': [ 1, 1, 1, 1, 1],
    'pair': [0, 1, 2, 3, 4],
    'impair': [4, 3, 2, 1, 0],
    'sum_b': [127.169725, 127.169725, 127.169725, 127.169725, 127.169725]
}

# Conversion en DataFrame
df_new = pd.DataFrame.from_dict(new_observation)
df_new

,const,pair,impair,sum_b
0,1,0,4,127.169725
1,1,1,3,127.169725
2,1,2,2,127.169725
3,1,3,1,127.169725
4,1,4,0,127.169725


In [79]:
# Si df_new n'a pas la colonne 'const', ajoutez-la
if 'const' not in df_new.columns:
    df_new = sm.add_constant(df_new)

# Faire la prédiction en utilisant le modèle
predicted_nombre_point= poisson_results.predict(df_new)
prediction_lambda = list(predicted_nombre_point)

# Création de la nouvelle observation avec les valeurs fournies
new_observation = {
    'const': [1, 1, 1, 1, 1],
    'pair': [0, 1, 2, 3, 4],
    'impair': [4, 3, 2, 1, 0],
    'sum_b': [127.169725, 127.169725, 127.169725, 127.169725, 127.169725],
    'estimation_lambda': prediction_lambda
}

# Conversion en DataFrame
df_new = pd.DataFrame.from_dict(new_observation)
df_new

,const,pair,impair,sum_b,estimation_lambda
0,1,0,4,127.169725,2.151410
1,1,1,3,127.169725,2.158668
2,1,2,2,127.169725,2.165952
3,1,3,1,127.169725,2.173259
4,1,4,0,127.169725,2.180592
